In [1]:
import numpy as np
import pandas as pd

In [2]:
temp_df = pd.read_csv('IMDB Dataset.csv')

In [3]:
df = temp_df.iloc[:10000]

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df.drop_duplicates(inplace=True)

C:\Windows\Temp\ipykernel_13120\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [6]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [7]:
# df['review'] = df['review'].apply(remove_tags)
df['review'] = df['review'].str.replace('<.*?>','',regex=True)

C:\Windows\Temp\ipykernel_13120\2373211115.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].str.replace('<.*?>','',regex=True)


In [8]:
df['review'] = df['review'].str.lower()

C:\Windows\Temp\ipykernel_13120\724319867.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].str.lower()


In [9]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

C:\Windows\Temp\ipykernel_13120\2826946130.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


In [10]:
df['review']

0       one reviewers mentioned watching 1 oz episode ...
1       wonderful little production. filming technique...
2       thought wonderful way spend time hot summer we...
3       basically there's family little boy (jake) thi...
4       petter mattei's "love time money" visually stu...
                              ...                        
9995    fun, entertaining movie wwii german spy (julie...
9996    give break. anyone say "good hockey movie"? kn...
9997    movie bad movie. watching endless series bad h...
9998    movie probably made entertain middle school, e...
9999    smashing film film-making. shows intense stran...
Name: review, Length: 9983, dtype: object

In [11]:
import gensim

In [12]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [13]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc) # returns a list of sentence tokens
    # print('raw_sent -> ',raw_sent)
    for sent in raw_sent:
        # print('simple_preprocess(sent) -> ',simple_preprocess(sent))
        story.append(simple_preprocess(sent))
    

In [42]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    max_vocab_size=3000
)

In [43]:
model.build_vocab(story)

In [44]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(2801281, 6212140)

In [45]:
import sys
sys.getsizeof(model)

56

In [46]:
len(model.wv.index_to_key)

933

In [47]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [48]:
document_vector(df['review'].values[0])

array([-0.06586011,  0.24295202,  0.07997464,  0.20196919, -0.04605724,
       -0.23689052,  0.02184579,  0.24773557, -0.14609498, -0.14328209,
       -0.07258582, -0.14413479, -0.01649496,  0.11166479, -0.04216485,
       -0.0664915 , -0.13124293, -0.01764555,  0.00235569, -0.00286807,
        0.00435414,  0.11546144,  0.0093271 , -0.16666621, -0.01289917,
       -0.18006776, -0.12015848,  0.02819988, -0.03345278,  0.07164934,
        0.00992511, -0.00281981,  0.07591115, -0.07406881, -0.16482304,
        0.07332365,  0.05357554, -0.18775782,  0.00342186, -0.20372812,
        0.13206975, -0.03380739,  0.08221685, -0.1860763 ,  0.16099371,
        0.12867482, -0.16609222, -0.16072392,  0.02659613,  0.11503024,
        0.04124623, -0.07467283, -0.24425402, -0.24081667,  0.06271303,
        0.06948408,  0.17964955,  0.23273286, -0.1199044 ,  0.01515296,
        0.07952923,  0.00976215,  0.00633209, -0.18113667, -0.15804687,
       -0.16340202,  0.1111347 , -0.10553107, -0.13715953,  0.14

In [49]:
from tqdm import tqdm

In [50]:
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████| 9983/9983 [00:15<00:00, 643.97it/s]


In [51]:
X = np.array(X)

In [52]:
X[0]

array([-0.06586011,  0.24295202,  0.07997464,  0.20196919, -0.04605724,
       -0.23689052,  0.02184579,  0.24773557, -0.14609498, -0.14328209,
       -0.07258582, -0.14413479, -0.01649496,  0.11166479, -0.04216485,
       -0.0664915 , -0.13124293, -0.01764555,  0.00235569, -0.00286807,
        0.00435414,  0.11546144,  0.0093271 , -0.16666621, -0.01289917,
       -0.18006776, -0.12015848,  0.02819988, -0.03345278,  0.07164934,
        0.00992511, -0.00281981,  0.07591115, -0.07406881, -0.16482304,
        0.07332365,  0.05357554, -0.18775782,  0.00342186, -0.20372812,
        0.13206975, -0.03380739,  0.08221685, -0.1860763 ,  0.16099371,
        0.12867482, -0.16609222, -0.16072392,  0.02659613,  0.11503024,
        0.04124623, -0.07467283, -0.24425402, -0.24081667,  0.06271303,
        0.06948408,  0.17964955,  0.23273286, -0.1199044 ,  0.01515296,
        0.07952923,  0.00976215,  0.00633209, -0.18113667, -0.15804687,
       -0.16340202,  0.1111347 , -0.10553107, -0.13715953,  0.14

In [53]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['sentiment'])

In [54]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [55]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [56]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [57]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7796695042563846